In [1]:
import json
import time
import re
from __future__ import division

In [20]:
listOfRestauranIds = []
with open('restaurantMapping.json') as json_data:
    loadingResJsons = json.load(json_data)
print len(loadingResJsons)
print type(loadingResJsons)

7730
<type 'dict'>


In [21]:
with open('user_ratings.json') as userRating_data:
    userRatingsJson = json.load(userRating_data)

userId = "1048504"

allReviewsForThisUser = userRatingsJson[userId]
print len(allReviewsForThisUser)
print len(userRatingsJson)

64
80621


In [4]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [5]:
with open('dataJsonFinal.json') as res_data:
    resJson = json.load(res_data)

@#$ Code for Timings@#$

In [6]:
def hoursAndMinsCalculate(startTime):
    if(':' in startTime):
#        print startTime
        hoursAndMins = re.search("""(.*?):(.*)""",startTime, re.I|re.S)
#        print "ya", hoursAndMins.group(1), type(hoursAndMins.group(1)), len(hoursAndMins.group(1))
        hours = hoursAndMins.group(1)
        mins = hoursAndMins.group(2)
        timings = str(int(hours)*100 + int(mins))
    else:
        timings = str(int(startTime)*100)
#    print "yup", timings
    return timings

In [7]:
def timingsCalculate(eachTime):
    timings = ""
    startTime = re.search("""(.*?)\sto""",eachTime,re.I|re.S).group(1)
    if("AM" in startTime):
        startTime = re.search("""(.*?)\sAM""",startTime, re.I|re.S).group(1)
        timings = hoursAndMinsCalculate(startTime) + "-"
    elif("PM" in startTime):
        startTime = re.search("""(.*?)\sPM""",startTime, re.I|re.S).group(1)
        tom = hoursAndMinsCalculate(startTime) 
        if( "12" not in tom ):
            timings = str(int(tom) + 1200) + "-"
        else:
            timings = tom + "-"
    elif("on" in startTime):
        timings = "1200-"

    endTime = re.search("""to\s(.*)""",eachTime,re.I|re.S).group(1)
    if("AM" in endTime):
        endTime = re.search("""(.*?)\sAM""",endTime, re.I|re.S).group(1)
        if (int(hoursAndMinsCalculate(endTime)) < 300):
            timings = timings + str(int(hoursAndMinsCalculate(endTime)) + 2400)
        else:
            timings = timings + hoursAndMinsCalculate(endTime)
    elif("PM" in endTime):
        endTime = re.search("""(.*?)\sPM""",endTime, re.I|re.S).group(1)
        timings = timings + str(int(hoursAndMinsCalculate(endTime)) + 1200)

    elif("on" in endTime):
        timings = timings + "1200"

    return timings

In [8]:
def checkClosedDay(whichDay):
    if("on" in whichDay):
        return 0
    elif("ue" in whichDay):
        return 1
    elif("ed" in whichDay):
        return 2
    elif("hu" in whichDay):
        return 3
    elif("ri" in whichDay):
        return 4
    elif("at" in whichDay):
        return 5
    else:
        return 6

In [9]:
def setClosedDays(eachString):
    ch = re.search("""\),(.*?)Closed""", eachString, re.I|re.S)
    if(ch == None):
        eachString = eachString.split('to')
        ch = re.search("""M,(.*?)Closed""", eachString[len(eachString)-1], re.I|re.S)

    daysClosed = re.sub("""\s""", "", ch.group(1), re.I|re.S)
    daysClosed = daysClosed.split(',')
    listOfTime = ["","","","","","",""]
    for eachTerm in daysClosed:
        if('-' in eachTerm):
            checkingDays = eachTerm.split('-')
            num = checkClosedDay(checkingDays[0])
            num1 = checkClosedDay(checkingDays[1])
            if(num < num1):
                while(num <= num1):
                    listOfTime[num] = "No"
                    num = num + 1
            else:
                while(num <= 6):
                    listOfTime[num] = "No"
                    num = num + 1
                num = 0
                while(num <= num1):
                    listOfTime[num] = "No"
                    num = num + 1

        else:
            num = checkClosedDay(eachTerm)
            listOfTime[num] = "No"
    return listOfTime

In [10]:
def setTime(eachTime): 
    listOfDaysTime = ["","","","","","",""]
    timings = ""    
    if(eachTime == "24 Hours"):
        listOfDaysTime = ["0-2400","0-2400","0-2400","0-2400","0-2400","0-2400","0-2400"]
        return listOfDaysTime
    #mon - 0, tue - 1, wed - 2, thurs - 3, fri -4, sat - 5
    if("Closed" in eachTime):
        listOfDaysTime = setClosedDays(eachTime)

    s = re.search("""(.*)\),(.*?)Closed""", eachTime, re.I|re.S)
    if(s != None):
        s = s.group(1)
        s = s + ")"
    else:
        s = eachTime

    s1 = re.findall("""(.*?)\)""", s, re.I|re.S)
    if(len(s1) != 0):
        for k in range(len(s1)):
            if(s1[k][:2] == ", "):
                s1[k] = s1[k][2:]

        for e in s1:
            searchBracketTime = re.search("""(.*?)\(""", e, re.I|re.S).group(1)
            searchBracketDays = re.search("""\((.*)""", e, re.I|re.S).group(1)
            searchBracketDays = searchBracketDays.split(',')
            detectComma = re.findall("""to""",searchBracketTime,re.I|re.S) #more than one time present
            numTimes = len(detectComma)
            if(numTimes == 1):
                timings = timingsCalculate(searchBracketTime)
            if(numTimes > 1):
                listOfTimings = searchBracketTime.split(',')
                for p in range(len(listOfTimings)):
                    if(listOfTimings[p][len(listOfTimings[p])-1] == ' '):
                        listOfTimings[p] = listOfTimings[p][:len(listOfTimings[p])-1]

                for m in range(len(listOfTimings)):
                    timings = timings + timingsCalculate(listOfTimings[m]) + '#'

            for eachTerm in searchBracketDays:
                if('-' in eachTerm):
                    checkingDays = eachTerm.split('-')
                    num = checkClosedDay(checkingDays[0])
                    num1 = checkClosedDay(checkingDays[1])
                    if(num < num1):
                        while(num <= num1):
                            listOfDaysTime[num] = timings
                            num = num + 1
                    else:
                        while(num <= 6):
                            listOfDaysTime[num] = timings
                            num = num + 1
                        num = 0
                        while(num <= num1):
                            listOfDaysTime[num] = timings
                            num = num + 1

                else:
                    num = checkClosedDay(eachTerm)
                    listOfDaysTime[num] = timings            

    else:
        detectComma = re.findall("""to""",eachTime,re.I|re.S) #more than one time present
        numTimes = len(detectComma)
        timings = ""
        if(numTimes == 1):
            timings = timingsCalculate(eachTime)
        if(numTimes > 1):
            listOfTimings = eachTime.split(',')
            for m in range(len(listOfTimings)):
                timings = timings + timingsCalculate(listOfTimings[m]) + '#'

    for t in range(len(listOfDaysTime)):
        if(listOfDaysTime[t] == ""):
            listOfDaysTime[t] = timings

                
    return listOfDaysTime

In [11]:
def checkIfResOpen(currentDay, currentTime, resTimings):
    w = setTime(resTimings)
    if(currentDay == "Monday"):
        timeCheck = w[0]
    elif(currentDay == "Tuesday"):
        timeCheck = w[1]
    elif(currentDay == "Wednesday"):
        timeCheck = w[2]
    elif(currentDay == "Thursday"):
        timeCheck = w[3]
    elif(currentDay == "Friday"):
        timeCheck = w[4]
    elif(currentDay == "Saturday"):
        timeCheck = w[5]
    else:
        timeCheck = w[6]
    if(timeCheck == "No"):
        return 0
    else:
        if(timeCheck[len(timeCheck)-1] == '#'):
            timeCheck = timeCheck[:len(timeCheck)-1]
        timeCheck = timeCheck.split('#')
        if(("pm" in currentTime) or ("PM" in currentTime)):
            currentTime = re.search("""(.*?)\sPM""",currentTime, re.I|re.S).group(1)
            cTime = int(hoursAndMinsCalculate(currentTime)) + 1200
        else:
            currentTime = re.search("""(.*?)\sAM""",currentTime, re.I|re.S).group(1)
            cTime = int(hoursAndMinsCalculate(currentTime))
        for eachOpening in timeCheck:
            eachOpen = eachOpening.split('-')
            openTime = int(eachOpen[0])
            endTime = int(eachOpen[1])
            if(cTime >= openTime):
                if(cTime <= endTime):
                    return 1

!@#Code For Timings!@#

In [12]:
def getPreferences(userId, numOfPref):
    #userId = "1048504"
    #Getting user preferences from dishes reviews 
    allReviewsForThisUser = userRatingsJson[userId]
    print len(allReviewsForThisUser)
    itemsLiked = []
    totalReviewsWritten = 0
    #print allReviewsForThisUser
    for j in range(len(allReviewsForThisUser)):
        totalReviewsWritten = totalReviewsWritten + allReviewsForThisUser[j][2]
        if((allReviewsForThisUser[j][2] < 5) and (allReviewsForThisUser[j][2] >= 3)):
            allReviewsForThisUser[j][1] = allReviewsForThisUser[j][1] + .01*(allReviewsForThisUser[j][2])
        elif(allReviewsForThisUser[j][2] >= 5):
            allReviewsForThisUser[j][1] = allReviewsForThisUser[j][1] + .02*(allReviewsForThisUser[j][2])
    allReviewsForThisUser.sort(key=lambda x: x[1], reverse=True)
    if(len(allReviewsForThisUser) > numOfPref): 
        return allReviewsForThisUser[:numOfPref] 
    else:
        return allReviewsForThisUser 

In [13]:
with open('final_data.json') as final_data:
    finalData = json.load(final_data)
dictOfDishes = {}
for eachResData in finalData:
    dictOfDishes[eachResData['resId']] = eachResData['dishes'].keys()
del finalData

In [14]:
with open('dish_ratings.json') as dish_ratings:
    dishRatings = json.load(dish_ratings)
dictOfDishesRatings = {}
for eachResRatingData in dishRatings:
    dictOfDishesRatings[eachResRatingData['resId']] = eachResRatingData['dishRatings']['ChunksWithRatings']
del dishRatings

In [ ]:
dishesIgnore = ['chutney', 'main course', 'starter', 'drink']

In [15]:
def recommend(userId, more, lat, lon):
    #lat = 12.937037
    #lon = 77.626488
    currentDay = "Monday"
    currentTime = "3:30 pm"
    #userId = "1048504"
    listOfFeasibleRestaurants = []
    detailsOfFeasibleRestaurants = []
    km4Restaurants = []
    for eachRes in resJson:
        resLatitude = float(eachRes['latitude'])
        resLongitude = float(eachRes['longitude'])
        resTimings = eachRes['timings']
        distFromYourLocation = haversine(lon, lat, resLongitude, resLatitude)
        try:
            if(distFromYourLocation < 3):
                if(resTimings != ""):
                    if(checkIfResOpen(currentDay, currentTime, resTimings) == 1):
                        detailsOfFeasibleRestaurants.append(eachRes)
                        listOfFeasibleRestaurants.append(str(eachRes['id']))
        except Exception,e:
            #print eachRes['id']
            #print repr(e)
            continue
    print len(listOfFeasibleRestaurants)
    dishesPrefer = getPreferences(userId, 20)
    dishesPreferred = [item[0] for item in dishesPrefer]
    print dishesPreferred
    suggestedRestaurantsWithCuisines = []#list of lists
    #print dictOfDishes.keys()
    listOfKeys = dictOfDishes.keys()
    for restaurantId in listOfFeasibleRestaurants:
        #print restaurantId
        if(restaurantId in dictOfDishes.keys()):
            availableDishes = dictOfDishes[restaurantId]
            checkIfDish = []
            for eachPreferredDish in dishesPreferred:
                checkIfDish = []
                if(eachPreferredDish in availableDishes):
                    try:
                        avgRatingForDish = dictOfDishesRatings[restaurantId][eachPreferredDish][-2]
                        checkIfDish.append(restaurantId)
                        checkIfDish.append(eachPreferredDish)
                        checkIfDish.append(avgRatingForDish)
                        suggestedRestaurantsWithCuisines.append(checkIfDish)
                    except Exception,e:
                        #print repr(e)
                        continue
    suggestedRestaurantsWithCuisines.sort(key=lambda x: x[2], reverse=True)
    return suggestedRestaurantsWithCuisines[(num*10):((num*10)+10)]
    

In [16]:
loadingResJsons['50416']

u'Xtreme Sports Bar, Bannerghatta Road'

In [17]:
#lat = 12.961009 #domlur
#lon = 77.637938
#lat = 12.952240 #marathalli
#lon = 77.700233

#lat = 12.937037 #Sony world signal
#lon = 77.626488
lat = 12.876535
lon = 77.566612
#userId - 1048504, 6508651 , 1313770 - cake, 15781419 - rice, 7639661, 30629221, 1570304 - pizza, 413925
num = 0
suggestedPlaces = recommend("1048504", num, lat, lon)
#print suggestedPlaces
print "\n"
print "Recommended Places to eat : "
for eachPlace in suggestedPlaces:
    restaurantName = loadingResJsons[eachPlace[0]]
    dishName = eachPlace[1]
    print "Place name : ", restaurantName, " \t dish :", dishName

73
64
[u'chicken', u'drink', u'rice', u'main course', u'potato', u'tea', u'chutney', u'spaghetti', u'mutton', u'kebab', u'cake', u'vada', u'vada pav', u'desert', u'sambar', u'chip', u'pizzas', u'dal makhani', u'chicken biryani', u'tikka']


Recommended Places to eat : 
Place name :  Kabab Mehal, JP Nagar  	 dish : cake
Place name :  Fine Flavors, Kanakapura Road  	 dish : potato
Place name :  Fine Flavors, Kanakapura Road  	 dish : desert
Place name :  Juicy Momos, JP Nagar  	 dish : drink
Place name :  Cafe Hops, JP Nagar  	 dish : cake
Place name :  Gowdara Mudde Mane, Kanakapura Road  	 dish : mutton
Place name :  Kabab Mehal, JP Nagar  	 dish : drink
Place name :  Goli Vada Pav No. 1, JP Nagar  	 dish : rice
Place name :  Rolls On Wheels, JP Nagar  	 dish : rice
Place name :  Rolls On Wheels, JP Nagar  	 dish : tikka


In [18]:
print userRatingsJson.keys()[60]

413925


In [19]:
print resJson[1]

{u'hasWifi': 0, u'name': u'Spicy North', u'hasBar': 0, u'hasDelivery': 1, u'isPureVeg': 1, u'longitude': u'77.5967761874', u'hasAC': 0, u'acceptsCreditCards': 1, u'smoking_area_flag': 2, u'takeaway_flag': 0, u'timings': u'12 Noon to 4 PM, 6:30 PM to 10:30 PM', u'cuisines': u'North Indian, Chinese', u'latitude': u'12.9165912603', u'buffet_details': u'', u'avgCostForTwo': 300, u'id': 50000, u'hasDineIn': 1, u'hasLiveMusic': 2}
